## Part 1

In [1]:
import findspark
findspark.init()
findspark.find()

'C:\\Spark\\spark-hadoop'

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]")\
.config("spark.driver.memory", "28g")\
.config("spark.jars.packages","databricks:spark-deep-learning:1.2.0-spark2.3-s_2.11")\
.appName("ch31_DeepLearningLappy-docs").getOrCreate()

#.config("spark.jars.packages","databricks:spark-deep-learning:1.2.0-spark2.3-s_2.11,databricks:tensorframes:0.5.0-s_2.11")\
# .config("spark.driver.memory", "8g")\
# .config("spark.executor.memory", "3g")\
# .config("spark.executor.cores", "4")\
# .config("spark.executor.instances", "10")\


#!/usr/bin/env bash

# This file is sourced when running various Spark programs.
# Copy it as spark-env.sh and edit that to configure Spark for your site.

# Options read in YARN client mode
#SPARK_EXECUTOR_INSTANCES="2" #Number of workers to start (Default: 2)
#SPARK_EXECUTOR_CORES="1" #Number of cores for the workers (Default: 1).
#SPARK_EXECUTOR_MEMORY="1G" #Memory per Worker (e.g. 1000M, 2G) (Default: 1G)
#SPARK_DRIVER_MEMORY="512M" #Memory for Master (e.g. 1000M, 2G) (Default: 512 Mb)
#SPARK_YARN_APP_NAME="spark" #The name of your application (Default: Spark)
#SPARK_YARN_QUEUE="default" #The hadoop queue to use for allocation requests (Default: default)
#SPARK_YARN_DIST_FILES="" #Comma separated list of files to be distributed with the job.
#SPARK_YARN_DIST_ARCHIVES="" #Comma separated list of archives to be distributed with the job.

In [3]:
sc = spark.sparkContext

In [4]:
spark

In [5]:
from sparkdl.image import imageIO
from pyspark.ml.image import ImageSchema

Using TensorFlow backend.


In [6]:
from pyspark.sql.functions import lit
from pyspark.sql.functions import expr
from pyspark.sql.types import DoubleType

In [7]:
# In[15]:
img_dir = "D:/Learn/GitRepos/Spark-The-Definitive-Guide/data/deep-learning-images"

In [8]:
tulips_df = ImageSchema.readImages(img_dir + "/tulips").withColumn("label", lit(1))
tulips_df.count()

30

In [9]:
# In[16]:
daisy_df = imageIO.readImagesWithCustomFn(img_dir + "/daisy", decode_f=imageIO.PIL_decode).withColumn("label", lit(0))
daisy_df.count()

30

In [10]:
# In[ ]:
tulips_train, tulips_test = tulips_df.randomSplit([0.6, 0.4])
daisy_train, daisy_test = daisy_df.randomSplit([0.6, 0.4])

In [11]:
# In[ ]:
train_images_df = tulips_train.unionAll(daisy_train).cache()
test_images_df = tulips_test.unionAll(daisy_test).cache()

In [12]:
print(train_images_df.count())
print(test_images_df.count())

41
19


## Part 2 

In [13]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline
from sparkdl import DeepImageFeaturizer

In [14]:
featurizer = DeepImageFeaturizer(inputCol="image", outputCol="features", modelName="InceptionV3")

lr = LogisticRegression(maxIter=20, regParam=0.05, elasticNetParam=0.3, labelCol="label")

p = Pipeline(stages=[featurizer, lr])

model = p.fit(train_images_df)    # train_images_df is a dataset of images and labels

In [15]:
# Inspect training error
df = model.transform(train_images_df.limit(10))

In [16]:
df.printSchema()

root
 |-- image: struct (nullable = true)
 |    |-- origin: string (nullable = true)
 |    |-- height: integer (nullable = false)
 |    |-- width: integer (nullable = false)
 |    |-- nChannels: integer (nullable = false)
 |    |-- mode: integer (nullable = false)
 |    |-- data: binary (nullable = false)
 |-- label: integer (nullable = false)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [17]:
predictionAndLabels = df.select("prediction", "label")

In [18]:
predictionAndLabels.count()

10

In [19]:
evaluator = MulticlassClassificationEvaluator(metricName="accuracy")
print("Training set accuracy = " + str(evaluator.evaluate(predictionAndLabels)))

Training set accuracy = 1.0


In [20]:
# Inspect testing error
df = model.transform(test_images_df.limit(10))
predictionAndLabels = df.select("prediction", "label")
print(predictionAndLabels.count())
evaluator = MulticlassClassificationEvaluator(metricName="accuracy")
print("Test set accuracy = " + str(evaluator.evaluate(predictionAndLabels)))

10
Test set accuracy = 1.0


## Part 3

In [22]:
import keras
from keras.applications import InceptionV3

model = InceptionV3(weights="imagenet")
model.save("D:/tmp/model-full-tmp.h5")

In [24]:
import os
#from os import listdir
def absoluteFilePaths(directory):
   for dirpath,_,filenames in os.walk(directory):
       for f in filenames:
           yield os.path.abspath(os.path.join(dirpath, f))

In [25]:
absoluteFilePaths(img_dir + "/tulips")

<generator object absoluteFilePaths at 0x0000020A164E5150>

In [27]:
tulips_files = [str(f) for f in absoluteFilePaths(img_dir + "/tulips")]  # make "local" file paths for images
daisy_files = [str(f) for f in absoluteFilePaths(img_dir + "/daisy")]  # make "local" file paths for images

In [28]:
tulips_files[:1]

['D:\\Learn\\GitRepos\\Spark-The-Definitive-Guide\\data\\deep-learning-images\\tulips\\3498663243_42b39b4185_m.jpg']

In [29]:
daisy_files[:1]

['D:\\Learn\\GitRepos\\Spark-The-Definitive-Guide\\data\\deep-learning-images\\daisy\\10555815624_dc211569b0.jpg']

In [37]:
import pyspark.ml.linalg as spla
import pyspark.sql.types as sptyp
import numpy as np

In [38]:
def CreateTrainImageUriandLabels(image_uris, label, cardinality):
  # Create image categorical labels (integer IDs)
  local_rows = []
  for uri in image_uris:
    label_inds = np.zeros(cardinality)
    label_inds[label] = 1.0
    one_hot_vec = spla.Vectors.dense(label_inds.tolist())
    _row_struct = {"uri": uri, "one_hot_label": one_hot_vec, "label": float(label)}
    row = sptyp.Row(**_row_struct)
    local_rows.append(row)

  image_uri_df = spark.createDataFrame(local_rows)
  return image_uri_df

In [39]:
label_cardinality = 2

In [40]:
tulips_uri_df = CreateTrainImageUriandLabels(tulips_files,1,label_cardinality)

In [41]:
daisy_uri_df = CreateTrainImageUriandLabels(daisy_files,0,label_cardinality)

In [42]:
tulips_uri_df.show(2,False)

+-----+-------------+---------------------------------------------------------------------------------------------------------+
|label|one_hot_label|uri                                                                                                      |
+-----+-------------+---------------------------------------------------------------------------------------------------------+
|1.0  |[0.0,1.0]    |D:\Learn\GitRepos\Spark-The-Definitive-Guide\data\deep-learning-images\tulips\3498663243_42b39b4185_m.jpg|
|1.0  |[0.0,1.0]    |D:\Learn\GitRepos\Spark-The-Definitive-Guide\data\deep-learning-images\tulips\3501996215_1c6d1a3386_n.jpg|
+-----+-------------+---------------------------------------------------------------------------------------------------------+
only showing top 2 rows



In [44]:
daisy_uri_df.show(2, False)

+-----+-------------+---------------------------------------------------------------------------------------------------------+
|label|one_hot_label|uri                                                                                                      |
+-----+-------------+---------------------------------------------------------------------------------------------------------+
|0.0  |[1.0,0.0]    |D:\Learn\GitRepos\Spark-The-Definitive-Guide\data\deep-learning-images\daisy\10555815624_dc211569b0.jpg  |
|0.0  |[1.0,0.0]    |D:\Learn\GitRepos\Spark-The-Definitive-Guide\data\deep-learning-images\daisy\10555826524_423eb8bf71_n.jpg|
+-----+-------------+---------------------------------------------------------------------------------------------------------+
only showing top 2 rows



In [45]:
#tulips_train, tulips_test, _ = tulips_uri_df.randomSplit([0.005, 0.005, 0.99]) 
# use larger training sets (e.g. [0.6, 0.4] for non-community edition clusters)
tulips_train, tulips_test = tulips_uri_df.randomSplit([0.8, 0.2]) 

In [46]:
#daisy_train, daisy_test, _ = daisy_uri_df.randomSplit([0.005, 0.005, 0.99])    
# use larger training sets (e.g. [0.6, 0.4] for non-community edition clusters)
daisy_train, daisy_test = daisy_uri_df.randomSplit([0.8, 0.2]) 

In [47]:
train_df = tulips_train.unionAll(daisy_train).cache()
test_df = tulips_test.unionAll(daisy_test).cache()

In [48]:
train_df.show(3, False)
test_df.show(3, False)

+-----+-------------+---------------------------------------------------------------------------------------------------------+
|label|one_hot_label|uri                                                                                                      |
+-----+-------------+---------------------------------------------------------------------------------------------------------+
|1.0  |[0.0,1.0]    |D:\Learn\GitRepos\Spark-The-Definitive-Guide\data\deep-learning-images\tulips\3498663243_42b39b4185_m.jpg|
|1.0  |[0.0,1.0]    |D:\Learn\GitRepos\Spark-The-Definitive-Guide\data\deep-learning-images\tulips\3501996215_1c6d1a3386_n.jpg|
|1.0  |[0.0,1.0]    |D:\Learn\GitRepos\Spark-The-Definitive-Guide\data\deep-learning-images\tulips\3502085373_edc2c36992_n.jpg|
+-----+-------------+---------------------------------------------------------------------------------------------------------+
only showing top 3 rows

+-----+-------------+----------------------------------------------------------

In [49]:
import PIL.Image
import numpy as np
from keras.applications.imagenet_utils import preprocess_input
from sparkdl.estimators.keras_image_file_estimator import KerasImageFileEstimator

In [50]:
def load_image_from_uri(local_uri):
  img = (PIL.Image.open(local_uri).convert('RGB').resize((299, 299), PIL.Image.ANTIALIAS))
  img_arr = np.array(img).astype(np.float32)
  img_tnsr = preprocess_input(img_arr[np.newaxis, :])
  return img_tnsr

In [55]:
train_df.select("uri").first()[0]

'D:\\Learn\\GitRepos\\Spark-The-Definitive-Guide\\data\\deep-learning-images\\tulips\\3498663243_42b39b4185_m.jpg'

In [56]:
test_load = load_image_from_uri(train_df.select("uri").first()[0])

In [67]:
test_load.shape

(1, 299, 299, 3)

In [65]:
estimatorT = KerasImageFileEstimator( inputCol="uri",
                                     outputCol="prediction",
                                     labelCol="one_hot_label",
                                     imageLoader=load_image_from_uri,
                                     kerasOptimizer='adam',
                                     kerasLoss='categorical_crossentropy', # why categorical_crossentropy
                                     modelFile="D:/tmp/model-full-tmp.h5", # local file path for model
                                     kerasFitParams = {"batch_size": 32, "verbose": 4}
                                   ) 

In [66]:
fittedT = estimatorT.fit(train_df)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 170.0 failed 1 times, most recent failure: Lost task 0.0 in stage 170.0 (TID 600, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "C:\Spark\spark-hadoop\python\lib\pyspark.zip\pyspark\worker.py", line 229, in main
  File "C:\Spark\spark-hadoop\python\lib\pyspark.zip\pyspark\worker.py", line 224, in process
  File "C:\Spark\spark-hadoop\python\lib\pyspark.zip\pyspark\serializers.py", line 372, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "C:\Users\KranthiKumar\AppData\Local\Temp\spark-abfc26a6-344c-4844-905b-8af3674c07f2\userFiles-3b8b69c5-5537-4d17-b3fe-0598415fb71f\databricks_spark-deep-learning-1.2.0-spark2.3-s_2.11.jar\sparkdl\estimators\keras_image_file_estimator.py", line 316, in _local_fit
  File "d:\l

In [58]:
estimator = KerasImageFileEstimator( inputCol="uri",
                                     outputCol="prediction",
                                     labelCol="one_hot_label",
                                     imageLoader=load_image_from_uri,
                                     kerasOptimizer='adam',
                                     kerasLoss='categorical_crossentropy', # why categorical_crossentropy
                                     modelFile="D:/tmp/model-full-tmp.h5" # local file path for model
                                   ) 

In [59]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [60]:
paramGrid = (
  ParamGridBuilder()
  .addGrid(estimator.kerasFitParams, [{"batch_size": 32, "verbose": 0},
                                      {"batch_size": 64, "verbose": 0}])
  .build()
)

In [61]:
bc = BinaryClassificationEvaluator(rawPredictionCol="prediction", labelCol="label" )
cv = CrossValidator(estimator=estimator, estimatorParamMaps=paramGrid, evaluator=bc, numFolds=2)

In [115]:
# Under the hood, each of the partitions is fully loaded in memory, which may be expensive.
# This ensure that each of the paritions has a small size.
# train_df = train_df.repartition(100)
# test_df = test_df.repartition(100)

In [62]:
cvModel = cv.fit(train_df)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 168.0 failed 1 times, most recent failure: Lost task 0.0 in stage 168.0 (TID 582, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "C:\Spark\spark-hadoop\python\lib\pyspark.zip\pyspark\worker.py", line 229, in main
  File "C:\Spark\spark-hadoop\python\lib\pyspark.zip\pyspark\worker.py", line 224, in process
  File "C:\Spark\spark-hadoop\python\lib\pyspark.zip\pyspark\serializers.py", line 372, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "C:\Users\KranthiKumar\AppData\Local\Temp\spark-abfc26a6-344c-4844-905b-8af3674c07f2\userFiles-3b8b69c5-5537-4d17-b3fe-0598415fb71f\databricks_spark-deep-learning-1.2.0-spark2.3-s_2.11.jar\sparkdl\estimators\keras_image_file_estimator.py", line 316, in _local_fit
  File "d:\l

In [117]:
# cvModel = cv.fit(train_df)

# #
# File "d:\learn\.virtualenvs\pyspark-lab\lib\site-packages\keras\engine\training_utils.py", line 137, in standardize_input_data
#     str(data_shape))
# ValueError: Error when checking target: expected predictions to have shape (1000,) but got array with shape (2,)


## part 4

In [ ]:
from pyspark.ml.image import ImageSchema
from sparkdl import DeepImagePredictor

image_df = ImageSchema.readImages(sample_img_dir)

predictor = DeepImagePredictor(inputCol="image", 
                               outputCol="predicted_labels", 
                               modelName="InceptionV3", 
                               decodePredictions=True, 
                               topK=10)

predictions_df = predictor.transform(image_df)

## part 5

In [ ]:
from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing.image import img_to_array, load_img
import numpy as np
import os
from pyspark.sql.types import StringType
from sparkdl import KerasImageFileTransformer

def loadAndPreprocessKerasInceptionV3(uri):
  # this is a typical way to load and prep images in keras
  image = img_to_array(load_img(uri, target_size=(299, 299)))  # image dimensions for InceptionV3
  image = np.expand_dims(image, axis=0)
  return preprocess_input(image)

transformer = KerasImageFileTransformer(inputCol="uri", outputCol="predictions",
                                        modelFile='/tmp/model-full-tmp.h5',  # local file path for model
                                        imageLoader=loadAndPreprocessKerasInceptionV3,
                                        outputMode="vector")

files = [os.path.abspath(os.path.join(dirpath, f)) for f in os.listdir("/data/myimages") if f.endswith('.jpg')]
uri_df = sqlContext.createDataFrame(files, StringType()).toDF("uri")

keras_pred_df = transformer.transform(uri_df)

## Part 6

In [ ]:
from sparkdl import KerasTransformer
from keras.models import Sequential
from keras.layers import Dense
import numpy as np

# Generate random input data
num_features = 10
num_examples = 100
input_data = [{"features" : np.random.randn(num_features).tolist()} for i in range(num_examples)]
input_df = sqlContext.createDataFrame(input_data)

# Create and save a single-hidden-layer Keras model for binary classification
# NOTE: In a typical workflow, we'd train the model before exporting it to disk,
# but we skip that step here for brevity
model = Sequential()
model.add(Dense(units=20, input_shape=[num_features], activation='relu'))
model.add(Dense(units=1, activation='sigmoid'))
model_path = "/tmp/simple-binary-classification"
model.save(model_path)

# Create transformer and apply it to our input data
transformer = KerasTransformer(inputCol="features", outputCol="predictions", modelFile=model_path)
final_df = transformer.transform(input_df)

## Part 7

In [ ]:
from keras.applications import InceptionV3
from sparkdl.udf.keras_image_model import registerKerasImageUDF

registerKerasImageUDF("inceptionV3_udf", InceptionV3(weights="imagenet"))

In [ ]:
registerKerasImageUDF("my_custom_keras_model_udf", "/tmp/model-full-tmp.h5")

In [ ]:
from keras.applications import InceptionV3
from sparkdl.udf.keras_image_model import registerKerasImageUDF

def keras_load_img(fpath):
    from keras.preprocessing.image import load_img, img_to_array
    import numpy as np
    img = load_img(fpath, target_size=(299, 299))
    return img_to_array(img).astype(np.uint8)

registerKerasImageUDF("inceptionV3_udf_with_preprocessing", InceptionV3(weights="imagenet"), keras_load_img)

In [ ]:
from pyspark.ml.image import ImageSchema

image_df = ImageSchema.readImages(sample_img_dir)
image_df.registerTempTable("sample_images")

In [ ]:
SELECT my_custom_keras_model_udf(image) as predictions from sample_images